In [1]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import datetime

end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(days=2)


btc_price = vbt.YFData.download(
        ["BTC-USD","ETH-USD"],
        missing_index='drop',
        start=start_time,
        end=end_time,
        interval="1m").get("Close")

C:\python_env\python311\Lib\site-packages\vectorbt\data\base.py:527: UserWarning: Symbols have mismatching index. Dropping missing data points.
  data = cls.align_index(data, missing=missing_index)


In [17]:
btc_price

symbol,BTC-USD,ETH-USD
Datetime,,
2024-09-17 16:29:00+00:00,60941.222656,2373.383057
2024-09-17 16:30:00+00:00,60944.250000,2372.990723
2024-09-17 16:31:00+00:00,60909.125000,2370.671387
2024-09-17 16:32:00+00:00,60813.871094,2367.607178
2024-09-17 16:33:00+00:00,60825.914062,2367.849609
...,...,...
2024-09-19 16:20:00+00:00,63258.597656,2455.176758
2024-09-19 16:21:00+00:00,63279.843750,2458.120605
2024-09-19 16:22:00+00:00,63294.718750,2456.455811


In [21]:
import talib

def custom_simple_indicator(close_df, rsi_window = 14):
    # rsi = talib.RSI(close, timeperiod = rsi_window)
    # # 遍历获取close_df的每一列
    # for col in close_df:
        # print(col)
    print(close_df)
    
    # rsi = vbt.RSI.run(close, window = rsi_window).rsi
    return close_df
ind = vbt.IndicatorFactory(
        class_name = "custom_simple_indicator",
        short_name = "csi",
        input_names = ["close"],
        param_names = ["rsi_window"],
        output_names = ["value"]
        ).from_apply_func(
                custom_simple_indicator,
                rsi_window = 14
                )

res = ind.run(
        btc_price,
        rsi_window = 21,
        keep_pd=True
        )
# res.value,type(res.value)

symbol                          BTC-USD      ETH-USD
Datetime                                            
2024-09-17 16:29:00+00:00  60941.222656  2373.383057
2024-09-17 16:30:00+00:00  60944.250000  2372.990723
2024-09-17 16:31:00+00:00  60909.125000  2370.671387
2024-09-17 16:32:00+00:00  60813.871094  2367.607178
2024-09-17 16:33:00+00:00  60825.914062  2367.849609
...                                 ...          ...
2024-09-19 16:20:00+00:00  63258.597656  2455.176758
2024-09-19 16:21:00+00:00  63279.843750  2458.120605
2024-09-19 16:22:00+00:00  63294.718750  2456.455811
2024-09-19 16:23:00+00:00  63263.730469  2454.669434
2024-09-19 16:24:00+00:00  63194.738281  2452.078125

[2371 rows x 2 columns]


In [25]:
import talib

def custom_simple_indicator(close:pd.DataFrame, rsi_window = 14):
    # rsi = talib.RSI(close, timeperiod = rsi_window)
    # # 遍历获取close_df的每一列
    ret = pd.DataFrame()
    for col in close.columns:
        ret[col] = talib.RSI(close[col], timeperiod = rsi_window)
    # print(close_df)
    
    # rsi = vbt.RSI.run(close, window = rsi_window).rsi
    return ret

ind = vbt.IndicatorFactory(
        class_name = "custom_simple_indicator",
        short_name = "csi",
        input_names = ["close"],
        param_names = ["rsi_window"],
        output_names = ["value"]
        ).from_apply_func(
                custom_simple_indicator,
                rsi_window = 14
                )

res = ind.run(
        btc_price,
        rsi_window = 21,
        keep_pd=True
        )
res.value

csi_rsi_window                    21           
symbol                       BTC-USD    ETH-USD
Datetime                                       
2024-09-17 16:29:00+00:00        NaN        NaN
2024-09-17 16:30:00+00:00        NaN        NaN
2024-09-17 16:31:00+00:00        NaN        NaN
2024-09-17 16:32:00+00:00        NaN        NaN
2024-09-17 16:33:00+00:00        NaN        NaN
...                              ...        ...
2024-09-19 16:20:00+00:00  53.733374  67.582516
2024-09-19 16:21:00+00:00  55.014879  69.892891
2024-09-19 16:22:00+00:00  55.912585  67.055167
2024-09-19 16:23:00+00:00  53.574021  64.121942
2024-09-19 16:24:00+00:00  48.802334  60.116571

[2371 rows x 2 columns]

In [31]:

def custom_indicator(close, rsi_window = 14, ma_window = 50):
    close_5m = close.resample("5T").last()
    rsi = vbt.RSI.run(close_5m, window = rsi_window).rsi
    rsi, _ = rsi.align(close, 
            broadcast_axis=0,
            method='ffill',
            join='right')

    close = close.to_numpy()
    rsi = rsi.to_numpy()
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()
    trend = np.where( rsi > 70, -1, 0)
    trend = np.where( (rsi < 30) & (close < ma), 1, trend)
    
    entries = np.where(trend == 1, True, False)
    exits = np.where(trend == -1, True, False)
    return entries,exits

ind = vbt.IndicatorFactory(
        class_name = "Combination",
        short_name = "comb",
        input_names = ["close"],
        param_names = ["rsi_window", "ma_window"],
        output_names = ["entries","exits"]
        ).from_apply_func(
                custom_indicator,
                rsi_window = 14,
                ma_window = 50,
                keep_pd=True
                )

res = ind.run(
        btc_price,
        rsi_window = 21,
        ma_window = 50
        )

print(res.entries.to_string())

pf = vbt.Portfolio.from_signals(btc_price, res.entries, res.exits)

print(pf.total_return())

comb_rsi_window                21        
comb_ma_window                 50        
symbol                    BTC-USD ETH-USD
Datetime                                 
2024-09-17 16:29:00+00:00   False   False
2024-09-17 16:30:00+00:00   False   False
2024-09-17 16:31:00+00:00   False   False
2024-09-17 16:32:00+00:00   False   False
2024-09-17 16:33:00+00:00   False   False
2024-09-17 16:34:00+00:00   False   False
2024-09-17 16:35:00+00:00   False   False
2024-09-17 16:36:00+00:00   False   False
2024-09-17 16:40:00+00:00   False   False
2024-09-17 16:41:00+00:00   False   False
2024-09-17 16:42:00+00:00   False   False
2024-09-17 16:45:00+00:00   False   False
2024-09-17 16:47:00+00:00   False   False
2024-09-17 16:48:00+00:00   False   False
2024-09-17 16:52:00+00:00   False   False
2024-09-17 16:53:00+00:00   False   False
2024-09-17 16:54:00+00:00   False   False
2024-09-17 16:56:00+00:00   False   False
2024-09-17 16:57:00+00:00   False   False
2024-09-17 16:58:00+00:00   False 